In [68]:
import pandas as pd
import numpy as np
import networkx as nx
from pyvis.network import Network
import json
import plotly.express as px
import plotly.graph_objects as go

In [69]:
with open('dict_entities.json', 'r') as f:
    db_dict = json.load(f)

In [70]:
db_dict

{'prod_017.PM152': {'cod_articolo': '017.PM152',
  'variante': 0,
  'costo': 10000.0,
  'data_riferimento': '2023-08-28',
  'categoria': 'Vetri',
  'descrizione': 'VETRO STRATIFICATO 33.1'},
 'prod_090.VST33': {'cod_articolo': '090.VST33',
  'variante': 0,
  'costo': 24.5,
  'data_riferimento': '2023-08-28',
  'categoria': 'Vetri',
  'descrizione': 'VETRO TRASPARENTE STRATIFICATO 3+3 F.G. ANTA INTELAIATA'},
 'prod_090.VST55': {'cod_articolo': '090.VST55',
  'variante': 0,
  'costo': 37.0,
  'data_riferimento': '2023-08-28',
  'categoria': 'Vetri',
  'descrizione': 'VETRO STRATIFICATO TRASPARENTE 5+5'},
 'prod_045.OB.P1T': {'cod_articolo': '045.OB.P1T',
  'variante': 0,
  'costo': 75.0,
  'data_riferimento': '2023-08-28',
  'categoria': 'Visive',
  'descrizione': "VETROCAMERA OBLO' PER PORTA 400X700 R100 TRASPARENTE S45"},
 'prod_052.MP.VT.POR.TR.45.04': {'cod_articolo': '052.MP.VT.POR.TR.45.04',
  'variante': 0,
  'costo': 112.0,
  'data_riferimento': '2023-08-28',
  'categoria': 'Visi

In [71]:
graph_db = nx.DiGraph()
for key in db_dict.keys():
    key_start = key.split('_')[0]

    if key_start == 'documento':
        cod_fornitore = db_dict[key]['cod_fornitore']
        azienda = db_dict[key]['azienda']
        importo = db_dict[key]['importo']
        id_documento = db_dict[key]['id_documento']
        fornitore = "fornitore_"+ str(cod_fornitore)
        if not graph_db.has_node(fornitore):
            graph_db.add_node(fornitore, type='fornitore')
        if not graph_db.has_node(azienda):
            graph_db.add_node(azienda, type='azienda')
        for prod in db_dict[key]['venduto']:
            codice_articolo = prod['codice_articolo']
            graph_db.add_edge(fornitore, azienda, type='materia', label= codice_articolo, name= codice_articolo, weight= 1)
        graph_db.add_edge(azienda, fornitore, type='money_neg', label= str(importo), name= str(importo), weight= importo)
        graph_db.add_edge(fornitore, azienda, type='document', label= id_documento, name= id_documento, weight= 1)

    elif key_start == 'preventivo':
        documento = db_dict[key]['id_documento']
        cliente = db_dict[key]['id_cliente']
        azienda = db_dict[key]['id_azienda']
        status = db_dict[key]['status']
        if not graph_db.has_node(cliente):
            graph_db.add_node(cliente, type='cliente')
        if not graph_db.has_node(azienda):
            graph_db.add_node(azienda, type='azienda')
        graph_db.add_edge(azienda, cliente, type='document', label= documento + "(" + status + ")", name= documento + "(" + status + ")", weight= 1)

    elif key_start == 'fattura':
        documento = db_dict[key]['id_documento']
        cliente = db_dict[key]['id_cliente']
        azienda = db_dict[key]['id_azienda']
        importo = db_dict[key]['importo']
        if not graph_db.has_node(cliente):
            graph_db.add_node(cliente, type='cliente')
        if not graph_db.has_node(azienda):
            graph_db.add_node(azienda, type='azienda')
        graph_db.add_edge(azienda, cliente, type='document', label= id_documento, name= id_documento, weight= 1)
        graph_db.add_edge(cliente, azienda, type='money_pos', label= str(importo), name= str(importo), weight= importo)
        for prod in db_dict[key]['venduto']:
            codice_articolo = prod['codice_articolo']
            graph_db.add_edge(azienda, cliente, type='materia', label= codice_articolo, name= codice_articolo, weight= 1)


In [72]:
graph_db_money = graph_db.copy()
for edge in graph_db.edges(data=True):
    if edge[2]['type'] not in ['money_pos','money_neg']:
        graph_db_money.remove_edge(edge[0], edge[1])
        
graph_db_materia = graph_db.copy()
for edge in graph_db.edges(data=True):
    if edge[2]['type'] != 'materia':
        graph_db_materia.remove_edge(edge[0], edge[1])

graph_db_document = graph_db.copy()
for edge in graph_db.edges(data=True):
    if edge[2]['type'] != 'document':
        graph_db_document.remove_edge(edge[0], edge[1])

In [73]:
graph_db_product = nx.DiGraph()
for key in db_dict.keys():
    key_start = key.split('_')[0]

    if key_start == 'documento':
        cod_fornitore = db_dict[key]['cod_fornitore']
        fornitore = "fornitore_"+ str(cod_fornitore)
        importo = db_dict[key]['importo']
        if not graph_db_product.has_node(fornitore):
            graph_db_product.add_node(fornitore, type='fornitore')
        for prod in db_dict[key]['venduto']:
            codice_articolo = prod['codice_articolo']
            articolo = "prod_" + codice_articolo
            if not graph_db_product.has_node(articolo):
                graph_db_product.add_node(articolo, type='materia', label= articolo, name= articolo, weight= 1)
        graph_db_product.add_edge(articolo, fornitore, type='money_neg', label= importo, name= importo, weight= importo)

    elif key_start == 'fattura':
        cliente = db_dict[key]['id_cliente']
        importo = db_dict[key]['importo']
        if not graph_db_product.has_node(cliente):
            graph_db_product.add_node(cliente, type='cliente')
        for prod in db_dict[key]['venduto']:
            codice_articolo = prod['codice_articolo']
            articolo = "prod_" + codice_articolo
            if not graph_db_product.has_node(articolo):
                graph_db_product.add_node(articolo, type='materia', label= codice_articolo, name= codice_articolo, weight= 1)
            graph_db_product.add_edge(cliente, articolo, type='money_pos', label= importo, name= importo, weight= importo)

In [74]:
graph_dict = {'all': graph_db, 'money': graph_db_money, 'materia': graph_db_materia, 'document': graph_db_document, 'prod': graph_db_product}

In [75]:
def create_graph(graph, name):
    # create pyvis network
    nt = Network(height="1024px", width="100%",
                 notebook=True, directed=True, 
                 neighborhood_highlight=True, select_menu=False, 
                 filter_menu=True, cdn_resources='in_line', bgcolor="#f4f4f4")
    #nt.from_nx(mrg)
    node_colors = {'fornitore': '#e0e098', 'azienda': '#76cc76', 'cliente': '#9d6ec9', 'materia': '#ff8c8c'}
    edge_colors = {'materia': 'black', 'money_neg': 'red', 'money_pos': 'green', 'document': 'blue'}
    for node in graph.nodes(data=True):
        node_name = node[0]
        node_type = node[1]['type']
        nt.add_node(node_name, title=node, color=node_colors[node_type])
    for edge in graph.edges(data=True):
        val = edge[2]['weight']
        edge_type = edge[2]['type']
        val = round(val, 2)
        label = edge[2]['label']
        try :
            label = float(label)
        except:
            label = edge[2]['label']
        if isinstance(label, float):
            label = round(label, 0)
        nt.add_edge(edge[0], edge[1], 
                    value=val, 
                    weight=val,
                    title=str(val),
                    label=str(label),
                    color=edge_colors[edge_type],
        )
    nt.force_atlas_2based(gravity=-50, overlap=1.0, central_gravity=0.005)
    nt.show_buttons(filter_=['physics'])#,'nodes'
    #nt.set_options("""
    #const options = {
    #  "borderWidth": 3,
    #  "borderWidthSelected": 5,
    #  "nodes": {
    #    "font": {
    #      "size": 42,
    #      "face": "verdana",
    #      "strokeWidth": 2
    #    },
    #    "size": 30
    #  },
    #  "physics": {
    #    "forceAtlas2Based": {
    #      "gravitationalConstant": -100,
    #      "centralGravity": 0.005,
    #      "springLength": 100,
    #      "avoidOverlap": 1
    #    },
    #    "minVelocity": 0.75,
    #    "solver": "forceAtlas2Based",
    #    "timestep": 1
    #  }
    #}
    #                """)
    nt.show("graph_analysis/"+name+".html")

In [76]:
for key in graph_dict.keys():
    create_graph(graph_dict[key], key)

graph_analysis/all.html
graph_analysis/money.html
graph_analysis/materia.html


graph_analysis/document.html
graph_analysis/prod.html


In [77]:
# for graph_money calculate in and out degree, betweenness and closeness, vitaility

def calculate_metrics(graph):
    in_degree = dict(graph.in_degree(weight='weight'))
    out_degree = dict(graph.out_degree(weight='weight'))
    betweenness = nx.betweenness_centrality(graph, weight='weight')
    closeness = nx.closeness_centrality(graph, distance='weight')
    #vitality = nx.closeness_vitality(graph, weight='weight')
    metrics = {}
    for node in graph.nodes():
        metrics[node] = {'in_degree': in_degree[node], 'out_degree': out_degree[node], 'betweenness': betweenness[node], 'closeness': closeness[node]}#, 'vitality': vitality[node]}
    return pd.DataFrame(metrics).T

In [78]:
df_metrics = calculate_metrics(graph_db)
df_metrics.to_csv('graph_analysis/metrics_money.csv')
df_metrics

,in_degree,out_degree,betweenness,closeness
fornitore_1964,47.500000,2.000000,0.073099,0.000049
mangini_3,1.000000,0.000000,0.000000,0.000049
mangini_1,130055.864647,56.500000,0.336257,0.000049
fornitore_1571,11040.000000,1.000000,0.000000,0.000019
mangini_2,4.000000,21762.120000,0.035088,0.210526
fornitore_597,8500.000000,1.000000,0.000000,0.000025
fornitore_5624,0.000000,1.000000,0.000000,0.000049
fornitore_3493,620.770000,1.000000,0.000000,0.000339
fornitore_584,1601.350000,1.000000,0.000000,0.000131
cliente_1,2.000000,50000.839623,0.000000,0.000063


In [82]:
df_metrics = calculate_metrics(graph_db_product)
df_metrics.to_csv('graph_analysis/metrics_product.csv')
df_metrics

,in_degree,out_degree,betweenness,closeness
fornitore_1964,249.500000,0.000000,0.000000,0.000061
prod_045.OB.P1T,0.000000,0.000000,0.000000,0.000000
fornitore_1571,11040.000000,0.000000,0.000000,0.000007
prod_017.PM001,60029.086094,11040.000000,0.006494,0.000007
fornitore_597,8500.000000,0.000000,0.000000,0.000004
prod_017.PM507,30000.621370,8500.000000,0.004329,0.000002
fornitore_5624,0.000000,0.000000,0.000000,0.000006
prod_090.VST55,0.000000,202.000000,0.000000,0.000000
fornitore_3493,620.770000,0.000000,0.000000,0.000026
prod_090.VST33,60151.090176,668.270000,0.021645,0.000019


In [159]:
# create a dataframe from graph_money
df_product = pd.DataFrame(graph_db_product.edges(data=True), columns=['source', 'target', 'data'])
df_product['type'] = df_product['data'].apply(lambda x: x['type'])
df_product['label'] = df_product['data'].apply(lambda x: x['label'])
df_product['label'] = df_product['label'].apply(lambda x: round(float(x), 2) if isinstance(x, str) else x)
money_neg_sum = df_product[df_product['type'] == 'money_neg']["label"].sum()
money_pos_sum = df_product[df_product['type'] == 'money_pos']["label"].sum()
df_product["label_percentage"] = df_product.apply(lambda x: x['label']/money_neg_sum if x['type'] == 'money_neg' else x['label']/money_pos_sum, axis=1)
df_product.drop(columns=['data'], inplace=True)
#df_money.to_csv('graph_analysis/df_money.csv')3
entities = list(set(df_product['source'].unique().tolist() + df_product['target'].unique().tolist()))
# assign a number to each entity
entity_dict = {entities[i]: i for i in range(len(entities))}
# assign category to each entity
entity_type = {entities[i]: graph_db_product.nodes[entities[i]]['type'] for i in range(len(entities))}
# assign a color to each entity type
entity_color = {'fornitore': 'red', 'cliente': 'blue', 'materia': '#bbffb2'}
# create the list of colors for each entity
entity_colors = [entity_color[entity_type[entity]] for entity in entities]
df_product['source_id'] = df_product['source'].apply(lambda x: entity_dict[x])
df_product['target_id'] = df_product['target'].apply(lambda x: entity_dict[x])
# assign a color to each type
df_product['color'] = df_product['type'].apply(lambda x: 'red' if x == 'money_neg' else 'green')
df_product

,source,target,type,label,label_percentage,source_id,target_id,color
0,prod_045.OB.P1T,fornitore_1964,money_neg,0.000000,0.000000,0,10,red
1,prod_017.PM001,fornitore_1571,money_neg,11040.000000,0.501553,4,17,red
2,prod_017.PM507,fornitore_597,money_neg,8500.000000,0.386160,3,1,red
3,prod_017.PM507,fornitore_5624,money_neg,0.000000,0.000000,3,18,red
4,prod_090.VST55,fornitore_1964,money_neg,202.000000,0.009177,7,10,red
5,prod_090.VST33,fornitore_3493,money_neg,620.770000,0.028202,22,12,red
6,prod_090.VST33,fornitore_1964,money_neg,47.500000,0.002158,22,10,red
7,prod_017.PM597,fornitore_584,money_neg,1601.350000,0.072750,20,19,red
8,prod_052.MP.VT.POR.TR.45.04,fornitore_1964,money_neg,0.000000,0.000000,6,10,red
9,cliente_0,prod_017.PM152,money_pos,30000.610208,0.085659,15,2,green


In [160]:
sources = df_product['source_id'].to_list()
targets = df_product['target_id'].to_list()
nodes = list(set(sources + targets))
values = df_product['label_percentage'].to_list()
values = [round(x, 2) for x in values]
edge_colors = df_product['color'].to_list()

In [161]:
fig = go.Figure(
    data=
    [
      go.Sankey(
        valueformat = ".2%",
        #valuesuffix = "%",
        #arrangement='snap',
        # Define nodes
        node = dict(
          pad = 15,
          thickness = 15,
          line = dict(color = "black", width = 0.5),
          label =  list(entity_dict.keys()),
          color =  entity_colors,
          #align="left",
        ),
        # Add links
        link = dict(
          arrowlen=10,
          source = sources,
          target =  targets,
          value =  values,
          #label =  df_product['label_percentage'].to_list(),
          color =  edge_colors
        )
      )
    ]
)

fig.update_layout(title_text="Products money flow",
                  font_size=10)
fig.show()

In [162]:
import urllib, json 

In [163]:
url = 'https://raw.githubusercontent.com/plotly/plotly.js/master/test/image/mocks/sankey_energy.json'
response = urllib.request.urlopen(url)
data = json.loads(response.read())

In [164]:
label =  data['data'][0]['node']['label']
label

["Agricultural 'waste'",
 'Bio-conversion',
 'Liquid',
 'Losses',
 'Solid',
 'Gas',
 'Biofuel imports',
 'Biomass imports',
 'Coal imports',
 'Coal',
 'Coal reserves',
 'District heating',
 'Industry',
 'Heating and cooling - commercial',
 'Heating and cooling - homes',
 'Electricity grid',
 'Over generation / exports',
 'H2 conversion',
 'Road transport',
 'Agriculture',
 'Rail transport',
 'Lighting & appliances - commercial',
 'Lighting & appliances - homes',
 'Gas imports',
 'Ngas',
 'Gas reserves',
 'Thermal generation',
 'Geothermal',
 'H2',
 'Hydro',
 'International shipping',
 'Domestic aviation',
 'International aviation',
 'National navigation',
 'Marine algae',
 'Nuclear',
 'Oil imports',
 'Oil',
 'Oil reserves',
 'Other waste',
 'Pumped heat',
 'Solar PV',
 'Solar Thermal',
 'Solar',
 'Tidal',
 'UK land based bioenergy',
 'Wave',
 'Wind']

In [165]:
data

{'data': [{'type': 'sankey',
   'domain': {'x': [0, 1], 'y': [0, 1]},
   'orientation': 'h',
   'valueformat': '.0f',
   'valuesuffix': 'TWh',
   'node': {'pad': 15,
    'thickness': 15,
    'line': {'color': 'black', 'width': 0.5},
    'label': ["Agricultural 'waste'",
     'Bio-conversion',
     'Liquid',
     'Losses',
     'Solid',
     'Gas',
     'Biofuel imports',
     'Biomass imports',
     'Coal imports',
     'Coal',
     'Coal reserves',
     'District heating',
     'Industry',
     'Heating and cooling - commercial',
     'Heating and cooling - homes',
     'Electricity grid',
     'Over generation / exports',
     'H2 conversion',
     'Road transport',
     'Agriculture',
     'Rail transport',
     'Lighting & appliances - commercial',
     'Lighting & appliances - homes',
     'Gas imports',
     'Ngas',
     'Gas reserves',
     'Thermal generation',
     'Geothermal',
     'H2',
     'Hydro',
     'International shipping',
     'Domestic aviation',
     'Internation